# Reproducing Numerical Results from "Fitted Value Iteration Methods for Bicausal Optimal Transport"

## Overview
This notebook reproduces the numerical experiments presented in the paper:

**"Fitted Value Iteration Methods for Bicausal Optimal Transport"**  
Erhan Bayraktar and Bingyan Han (2023)  
[arXiv:2306.12658](https://arxiv.org/abs/2306.12658)

We specifically aim to replicate:

- **Table 1**: The LP method with backward induction for bicausal optimal transport.
- **Table 2**: The adapted Sinkhorn method for bicausal optimal transport.

Additionally, we compare the results and computational speed of these methods against our own implementation.

In [1]:
import numpy as np
import os
import sys
import warnings
import time

# Define paths to relevant modules
measure_sampling_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Measure_sampling')
aot_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AOT_numerics')
trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees')
Benchmark_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Benchmark_value_Gausian')
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add paths to sys.path
for path in [measure_sampling_path, aot_path, awd_trees_path, trees_path, Benchmark_path]:
    if path not in sys.path:
        sys.path.append(path)

# Import necessary modules
from Gen_Path_and_AdaptedTrees import *
from mainfunctions import *
from measure import *
from normal_ot import *
from FVI_bench import *
from Extract_Sample_path_AOT import extract_sample_paths
from Tree_Node import *
from TreeAnalysis import *
from TreeVisualization import *
from Save_Load_trees import *
from Tree_AWD_utilities import *
from Gurobi_AOT import *
from Nested_Dist_Algo import compute_nested_distance
from Build_trees_from_paths import *
from Comp_AWD2_Gaussian import build_mean_and_cov, adapted_wasserstein_squared

# Suppress sklearn warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Compute backward using POT solver

In [2]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

# --- Parameters that do not change ---
np.random.seed(12345)
N_INSTANCE = 10
N_BRANCH = 2
x_vol = 1.0
y_vol = 0.5
x_init = 1.0
y_init = 2.0

def cost(x, y):
    return (x[0] - y[0]) ** 2

# We'll collect final results for each T in a list of rows:
results_table = []

# Loop over T from 1 to 7
for T_val in range(1, 4):

    print(f"\n========== Running for T = {T_val} ==========")

    # Define parameters
    a, b = 1, 2
    var_a, var_b = 1**2, 0.5**2
    t = T_val

        
    # Build mean and covariance matrices for both processes
    a_vec, A_mat = build_mean_and_cov(t, mean_val=a, var_factor=var_a)
    b_vec, B_mat = build_mean_and_cov(t, mean_val=b, var_factor=var_b)

    # Compute adapted Wasserstein squared distance
    bench_val = adapted_wasserstein_squared(a_vec, A_mat, b_vec, B_mat)
    
    # Prepare storage for repeated runs
    final_result_bw = np.zeros(N_INSTANCE)
    final_result_pot = np.zeros(N_INSTANCE)
    bw_times = []
    pot_times = []
    
    # Build the graph for the Markov chain
    g = Graph(T_val + 1)
    for t in range(T_val):
        g.addEdge(t, t + 1)
    
    # Start measuring time for all N_INSTANCE runs (optional, not necessarily used below)
    overall_t0 = time.time()
    
    for n_ins in range(N_INSTANCE):
        print(f"\n  Instance {n_ins+1}/{N_INSTANCE} for T={T_val}")
        
        # Generate random measures mu, nu
        mu, supp_mu = rand_tree_binom(T_val, init=x_init, vol=x_vol, N_leaf=N_BRANCH, in_size=200)
        nu, supp_nu = rand_tree_binom(T_val, init=y_init, vol=y_vol, N_leaf=N_BRANCH, in_size=200)

        # Extract sample paths
        sample_path_x, weight_x = extract_sample_paths(mu, T_val, x_init)
        sample_path_y, weight_y = extract_sample_paths(nu, T_val, y_init)

        # Build trees
        x_root = build_tree_from_paths(sample_path_x, weight_x)
        y_root = build_tree_from_paths(sample_path_y, weight_y)

        # ----------------
        # 1) POT Solver
        # ----------------
        print("    Computing adapted OT using POT solver...")
        max_depth = get_depth(x_root)
        start_time_pot = time.time()
        distance_pot = compute_nested_distance(
            x_root, y_root, max_depth,
            method="solver_lp_pot", return_matrix=False,
            lambda_reg=0, power=2
        )
        elapsed_time_pot = time.time() - start_time_pot
        pot_times.append(elapsed_time_pot)
        
        # Subtract (x_init - y_init)^2 to be consistent with your example
        distance_pot_adjusted = distance_pot - (x_init - y_init) ** 2
        final_result_pot[n_ins] = distance_pot_adjusted
        
        print(f"    POT Distance: {distance_pot_adjusted:.4f}  Time: {elapsed_time_pot:.2f}s")

        # ----------------
        # 2) Backward Induction (BW)
        # ----------------
        cost_funs = [[[t], cost] for t in range(T_val + 1)]
        start_time_bw = time.time()
        BW_v1, _ = solve_dynamic(
            cost_funs, mu, nu, supp_mu, supp_nu, g,
            outputflag=0, method='pot'
        )
        elapsed_time_bw = time.time() - start_time_bw
        bw_times.append(elapsed_time_bw)

        # Also subtract (x_init - y_init)^2
        BW_v1_adjusted = BW_v1[0] - (x_init - y_init) ** 2
        final_result_bw[n_ins] = BW_v1_adjusted
        
        print(f"    BW Distance: {BW_v1_adjusted:.4f}  Time: {elapsed_time_bw:.2f}s")

    # Summarize the results for T_val
    mean_bw  = np.mean(final_result_bw)
    std_bw   = np.std(final_result_bw)
    mean_pot = np.mean(final_result_pot)
    std_pot  = np.std(final_result_pot)
    
    mean_bw_time  = np.mean(bw_times)
    mean_pot_time = np.mean(pot_times)
    
    overall_elapsed = time.time() - overall_t0
    
    print("\n  Summary for T =", T_val)
    print("  BW mean (std)  =", f"{mean_bw:.4f} ({std_bw:.4f})")
    print("  POT mean (std) =", f"{mean_pot:.4f} ({std_pot:.4f})")
    print("  BW avg time    =", f"{mean_bw_time:.3f}")
    print("  POT avg time   =", f"{mean_pot_time:.3f}")
    print("  Overall time   =", f"{overall_elapsed:.3f}s")

    # Store row in our results table.
    # Example: we show '2x distance' in one column per method, plus time columns.
    # If you prefer not to multiply by 2, just remove the "2*" below.
    row = [
        T_val,
        bench_val,
        f"{mean_bw:.3f} ({std_bw:.3f})",  # Their implentation
        f"{mean_pot:.3f} ({std_pot:.3f})",# My implementation
        f"{mean_bw_time:.3f}",               # Their time
        f"{mean_pot_time:.3f}"               # My time
    ]

    results_table.append(row)


========== Running for T = 1 ==========

  Instance 1/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.2327  Time: 0.00s
Set parameter Username
Set parameter LicenseID to value 2604970
Academic license - for non-commercial use only - expires 2026-01-03
    BW Distance: 1.2327  Time: 0.03s

  Instance 2/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.2817  Time: 0.00s
    BW Distance: 1.2817  Time: 0.00s

  Instance 3/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.0841  Time: 0.00s
    BW Distance: 1.0841  Time: 0.00s

  Instance 4/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.3221  Time: 0.00s
    BW Distance: 1.3221  Time: 0.00s

  Instance 5/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.1084  Time: 0.00s
    BW Distance: 1.1084  Time: 0.00s

  Instance 6/10 for T=1
    Computing adapted OT using POT solver...
    POT Distance: 1.1293  Time: 

In [3]:
# ---------------------------------------------------------
# Generate a pandas DataFrame with all results
# ---------------------------------------------------------
columns = ["T", "Actual", "Their estimated mean (std)", "My estimated mean (std)", "Their average time", "My average time"]
df = pd.DataFrame(results_table, columns=columns)

print("\nFinal Results Table:")
print(df)


Final Results Table:
   T  Actual Their estimated mean (std) My estimated mean (std)   
0  1    1.25              1.217 (0.102)           1.217 (0.102)  \
1  2    2.75              2.643 (0.161)           2.643 (0.161)   
2  3    4.50              4.227 (0.418)           4.227 (0.418)   

  Their average time My average time  
0              0.005           0.000  
1              0.004           0.002  
2              0.011           0.006  


## Compute backward using Sinkhorn Solver

In [4]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------------------------------------
# 1. Setup: define the EPS/lambda_for_eps values per T
# ----------------------------------------------------------
eps_values = {
    1: 0.1,  2: 0.1,  3: 0.1,  4: 0.1,  5: 0.1,
    6: 0.2,  7: 0.4,  8: 0.6,  9: 0.8, 10: 1.0
}
lambda_values = {
    1: 10,   2: 10,   3: 10,   4: 10,   5: 10,
    6: 5,    7: 2.5,  8: 1.667,9: 1.25,10: 1.0
}

# ----------------------------------------------------------
# 2. Other global parameters (unchanged)
# ----------------------------------------------------------
np.random.seed(12345)

N_INSTANCE = 10
N_BRANCH = 2

x_vol = 1.0
y_vol = 0.5
x_init = 1.0
y_init = 2.0

def cost_f_scalar_2(x, y):
    return np.abs(x - y)**2

# This list will hold our final summary results for each T
results_table_Sinkhorn = []

# ----------------------------------------------------------
# 3. Loop over T = 1 to 10
# ----------------------------------------------------------
for T_val in range(1, 4):
    print(f"\n================== Running for T = {T_val} ==================")

        # Define parameters
    a, b = 1, 2
    var_a, var_b = 1**2, 0.5**2
    t = T_val

        
    # Build mean and covariance matrices for both processes
    a_vec, A_mat = build_mean_and_cov(t, mean_val=a, var_factor=var_a)
    b_vec, B_mat = build_mean_and_cov(t, mean_val=b, var_factor=var_b)

    # Compute adapted Wasserstein squared distance
    bench_val = adapted_wasserstein_squared(a_vec, A_mat, b_vec, B_mat)

    # Get the EPS and lambda_for_eps for this T
    EPS_current = eps_values[T_val]
    lambda_current = lambda_values[T_val]
    print(f"  Using EPS = {EPS_current}, lambda_for_eps = {lambda_current}")

    # Prepare arrays to store repeated-run results for this T
    final_result_bw = np.zeros(N_INSTANCE)
    final_result_Sinkhorn = np.zeros(N_INSTANCE)
    bw_times = []
    Sinkhorn_times = []

    # Start measuring total time for T_val (optional)
    overall_t0 = time.time()

    for n_ins in range(N_INSTANCE):
        print(f"\n  Instance {n_ins+1}/{N_INSTANCE} for T = {T_val}")

        # ---------------------------------------------------------
        # 1) Generate random measures mu, nu
        # ---------------------------------------------------------
        mu, supp_mu = rand_tree_binom(T_val, init=x_init, vol=x_vol, N_leaf=N_BRANCH, in_size=200)
        nu, supp_nu = rand_tree_binom(T_val, init=y_init, vol=y_vol, N_leaf=N_BRANCH, in_size=200)

        # Extract sample paths
        sample_path_x, weight_x = extract_sample_paths(mu, T_val, x_init)
        sample_path_y, weight_y = extract_sample_paths(nu, T_val, y_init)

        # Build trees
        x_root = build_tree_from_paths(sample_path_x, weight_x)
        y_root = build_tree_from_paths(sample_path_y, weight_y)

        # ---------------------------------------------------------
        # 2) Compute adapted optimal transport using Sinkhorn solver
        #    (via 'compute_nested_distance' with method="Sinkhorn")
        # ---------------------------------------------------------
        max_depth = get_depth(x_root)
        print("    Computing adapted OT using Sinkhorn solver...")

        start_time_Sinkhorn = time.time()
        distance_Sinkhorn = compute_nested_distance(
            x_root, y_root, max_depth,
            method="Sinkhorn", return_matrix=False,
            lambda_reg=lambda_current,  # <-- Use the T-specific lambda
            power=2
        )
        elapsed_time_Sinkhorn = time.time() - start_time_Sinkhorn
        Sinkhorn_times.append(elapsed_time_Sinkhorn)

        # Subtract (x_init - y_init)**2 for consistency
        distance_Sinkhorn_adj = distance_Sinkhorn - (x_init - y_init)**2
        final_result_Sinkhorn[n_ins] = distance_Sinkhorn_adj

        print(f"    Sinkhorn Solver Distance: {distance_Sinkhorn_adj:.4f}, "
              f"Computation Time: {elapsed_time_Sinkhorn:.2f}s")

        # ---------------------------------------------------------
        # 3) "BW" approach (your bicausal Sinkhorn)
        #
        #    - Re-collect measure arrays for your custom sinkhorn_bicausal_markov
        #    - Build cost_mats_2 and run 'sinkhorn_bicausal_markov'
        # ---------------------------------------------------------
        x_list, mu_list = get_meas_for_sinkhorn(mu, supp_mu, T_val + 1)
        y_list, nu_list = get_meas_for_sinkhorn(nu, supp_nu, T_val + 1)

        # Build indexing structures
        ind_tot = get_full_index_markov(nu_list)
        ind_next_l = get_start_next_indices(ind_tot)
        nu_joint_prob = get_joint_prob(nu_list, ind_tot, T_val - 1)

        # Construct cost matrices for each time step
        cost_mats_2 = []
        for t in range(T_val + 1):
            cmh_2 = np.zeros((len(x_list[t]), len(y_list[t])), dtype=np.float64)
            for i in range(len(x_list[t])):
                for j in range(len(y_list[t])):
                    # Exponential of negative cost / EPS
                    cmh_2[i, j] = np.exp(-1.0 / EPS_current * cost_f_scalar_2(x_list[t][i], y_list[t][j]))
            cost_mats_2.append(cmh_2)

        n_list = [len(x_list[i]) for i in range(T_val + 1)]
        m_list = [len(y_list[i]) for i in range(T_val + 1)]

        print("    n_list:", n_list)
        print("    m_list:", m_list)

        start_time_bw = time.time()
        val_sink_2 = sinkhorn_bicausal_markov(
            mu_list, nu_list, cost_mats_2, n_list, m_list,
            eps_stop=1e-4, max_iter=1000, reshape=True, outputflag=0
        )
        elapsed_time_bw = time.time() - start_time_bw
        bw_times.append(elapsed_time_bw)

        # Multiply by EPS to get the cost, then subtract the shift
        sink_bc_v2 = val_sink_2 * EPS_current
        BW_v1 = sink_bc_v2 - (x_init - y_init)**2
        final_result_bw[n_ins] = BW_v1

        print(f"    BW bicausal Sinkhorn Value: {BW_v1:.4f}, "
              f"Computation Time: {elapsed_time_bw:.2f}s")

    # ---------------------------------------------------------
    # Summarize results for this T_val
    # ---------------------------------------------------------
    mean_bw       = np.mean(final_result_bw)
    std_bw        = np.std(final_result_bw)
    mean_Sinkhorn = np.mean(final_result_Sinkhorn)
    std_Sinkhorn  = np.std(final_result_Sinkhorn)

    mean_bw_time       = np.mean(bw_times)
    mean_Sinkhorn_time = np.mean(Sinkhorn_times)

    overall_elapsed = time.time() - overall_t0

    print(f"\n  Summary for T = {T_val}")
    print(f"  BW mean (std)        = {mean_bw:.4f} ({std_bw:.4f})")
    print(f"  Sinkhorn mean (std)  = {mean_Sinkhorn:.4f} ({std_Sinkhorn:.4f})")
    print(f"  BW avg time          = {mean_bw_time:.3f}")
    print(f"  Sinkhorn avg time    = {mean_Sinkhorn_time:.3f}")
    print(f"  Overall time so far  = {overall_elapsed:.3f}s")
    print(EPS_current)

    # ---------------------------------------------------------
    # Append a row to our results_table.
    #   If you want "twice" the distances, multiply by 2 below.
    # ---------------------------------------------------------
    row = [
        T_val,
        bench_val,
        f"{mean_bw:.3f} ({std_bw:.3f})",              #  BW distance
        f"{mean_Sinkhorn:.3f} ({std_Sinkhorn:.3f})",  #  Sinkhorn distance
        f"{mean_bw_time:.3f}",                            # BW time
        f"{mean_Sinkhorn_time:.3f}"                       # Sinkhorn time
    ]
    results_table_Sinkhorn.append(row)


================== Running for T = 1 ==================
  Using EPS = 0.1, lambda_for_eps = 10

  Instance 1/10 for T = 1
    Computing adapted OT using Sinkhorn solver...
    Sinkhorn Solver Distance: 1.2327, Computation Time: 0.01s
    n_list: [1, 2]
    m_list: [1, 2]
    BW bicausal Sinkhorn Value: 1.2240, Computation Time: 0.01s

  Instance 2/10 for T = 1
    Computing adapted OT using Sinkhorn solver...
    Sinkhorn Solver Distance: 1.2816, Computation Time: 0.02s
    n_list: [1, 2]
    m_list: [1, 2]
    BW bicausal Sinkhorn Value: 1.2647, Computation Time: 0.03s

  Instance 3/10 for T = 1
    Computing adapted OT using Sinkhorn solver...
    Sinkhorn Solver Distance: 1.0817, Computation Time: 0.18s
    n_list: [1, 2]
    m_list: [1, 2]
    BW bicausal Sinkhorn Value: 1.0612, Computation Time: 0.12s

  Instance 4/10 for T = 1
    Computing adapted OT using Sinkhorn solver...
    Sinkhorn Solver Distance: 1.3220, Computation Time: 0.03s
    n_list: [1, 2]
    m_list: [1, 2]
    

In [5]:
# ------------------------------------------------------------------
# Build a DataFrame and display/save the final results
# ------------------------------------------------------------------
columns = ["T", "Actual", "Their estimated mean (std)", "My estimated mean (std)", "Their average time", "My average time"]
df = pd.DataFrame(results_table_Sinkhorn, columns=columns)

print("\nFinal Results Table:")
print(df)


Final Results Table:
   T  Actual Their estimated mean (std) My estimated mean (std)   
0  1    1.25              1.201 (0.103)           1.216 (0.102)  \
1  2    2.75              2.620 (0.164)           2.636 (0.165)   
2  3    4.50              4.173 (0.422)           4.196 (0.418)   

  Their average time My average time  
0              0.048           0.050  
1              0.174           0.171  
2              0.836           0.747  


## Exemple of use of the solve_dynamic methode from AOT_numerics

In [11]:
# ----- Parameters -----
num_paths = 20          # Number of sample paths for each measure.
time_steps = 3          # Total time steps (each path will have time_steps+1 entries, including time 0).
scale_x = 1.0 
scale_y = 2             # Scale parameter for Brownian motion.
use_weights = 1          # Set to 1 to compute weights via empirical clustering.

# ----- Generate Sample Paths and Empirical Measures -----
# Generate Brownian motion samples for μ and ν.
sample_paths_x, sample_time_y = generate_brownian_motion(num_paths, time_steps, scale_x, return_time=True)
sample_paths_y, sample_time_Y = generate_brownian_motion(num_paths, time_steps, scale_y, return_time=True)

# Compute empirical measures using k-means clustering.
# The empirical_k_means_measure function returns a tuple: (support array, weights).
new_sample_paths_x, new_weights_x = empirical_k_means_measure(sample_paths_x, use_weights=use_weights)
new_sample_paths_y, new_weights_y = empirical_k_means_measure(sample_paths_y, use_weights=use_weights)

print("Empirical Measure for μ (support):", new_sample_paths_x.shape)
print("Empirical Measure for ν (support):", new_sample_paths_y.shape)

# ----- Build the Graph -----
# We assume the columns of new_sample_paths_x and new_sample_paths_y correspond to time steps.
T_h = new_sample_paths_x.shape[1]  # Number of time steps in the empirical measure.
g = Graph(T_h)
for t in range(T_h - 1):
    g.addEdge(t, t + 1)

# ----- Define the Measure and Support Functions -----
# For a static (non-adapted) measure, we can simply ignore the conditioning.
def mu(node, x_parents):
    # Return the marginal at the given time (node) as a 2D array (support) and the weights.
    # Here, new_sample_paths_x is assumed to be of shape (k, T_h) where k is the number of clusters.
    support = new_sample_paths_x[:, node:node+1]  # Extract column 'node' as a (k,1) array.
    return [support, new_weights_x]

def supp_mu(node_list):
    # Given a list of node indices, return the corresponding columns from new_sample_paths_x.
    if len(node_list) == 0:
        # If there are no nodes, return an empty array with the right number of rows.
        return np.empty((new_sample_paths_x.shape[0], 0))
    return new_sample_paths_x[:, node_list]

def nu(node, x_parents):
    support = new_sample_paths_y[:, node:node+1]
    return [support, new_weights_y]

def supp_nu(node_list):
    if len(node_list) == 0:
        return np.empty((new_sample_paths_y.shape[0], 0))
    return new_sample_paths_y[:, node_list]

# ----- Define the Squared Cost Function -----
# This cost function computes the squared difference.
def square_cost(x, y):
    return (x[0] - y[0])**2

# Build the cost list for solve_dynamic.
# For each time node t, we associate a cost function that depends only on that node.
cost_list = [[[t], square_cost] for t in range(T_h)]

# ----- Solve the Adapted Optimal Transport Problem -----
start_time = time.time()
# solve_dynamic returns a list of optimal values and additional coupling information.
out_vals, opt_info = solve_dynamic(cost_list, mu, nu, supp_mu, supp_nu, g, outputflag=0, method='pot')
elapsed_time = time.time() - start_time

# ----- Display the Results -----
print("Optimal dynamic transport values:", out_vals)
print("Elapsed time (seconds):", elapsed_time)

Empirical Measure for μ (support): (11, 3)
Empirical Measure for ν (support): (10, 3)
Optimal dynamic transport values: [4.8173983404291025]
Elapsed time (seconds): 0.9188611507415771
